In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import calendar
import os
import funciones
import sys
import logging

# Importar configuración
from funciones_principales import (
    configurar_rutas, cargar_datos, limpiar_formato_numerico_excel, limpiar_dataframe_excel
)

def configurar_rutas():
    """
    Configura las rutas y nombres de archivos necesarios para el procesamiento.
    
    Returns:
        dict: Diccionario con rutas y nombres de archivos
    """
    config = {
        'path': RUTAS['BASE'],
        'archivo_ventas': ARCHIVOS_ENTRADA['VENTAS'],
        'archivo_clientes': ARCHIVOS_ENTRADA['CLIENTES'],
        'archivo_costos': ARCHIVOS_ENTRADA['COSTOS'],
        'archivo_marca_referencia': ARCHIVOS_ENTRADA['MARCA_REFERENCIA'],
        'archivo_metas': ARCHIVOS_ENTRADA['METAS'],

        'archivo_promotora': ARCHIVOS_ENTRADA['CLIENTE_PROMOTORA'],
        'archivo_makro': ARCHIVOS_ENTRADA['CLIENTE_MAKRO'],
        'archivo_farmatodo': ARCHIVOS_ENTRADA['CLIENTE_FARMATODO'],
        

        'archivo_tabla_sell_out': ARCHIVOS_CONSOLIDADOS['CONSOLIDADO_SELL_OUT'],
        'archivo_tabla_sell_out_ciudades': ARCHIVOS_CONSOLIDADOS['CONSOLIDADO_SELL_OUT_CIUDADES'],
        'archivo_ventas_consolidado': ARCHIVOS_CONSOLIDADOS['VENTAS_CONSOLIDADOS'],
        'archivo_costos_consolidado': ARCHIVOS_CONSOLIDADOS['COSTOS_CONSOLIDADOS'],
        
        'ultima_fecha': FECHAS['ULTIMA_FECHA_CONSOLIDADO']
    }
    
    sys.path.append(obtener_ruta_completa('codigos'))
    aplicar_configuracion_pandas()
    
    return config


def cargar_datos(config):
    """
    Carga todas las tablas necesarias desde los archivos Excel.
    
    Args:
        config (dict): Configuración con rutas y nombres de archivos
        
    Returns:
        dict: Diccionario con todas las tablas cargadas
    """
    datos = {}
    ruta_datos = obtener_ruta_completa('entrada')
    ruta_datos_consolidado = obtener_ruta_completa('consolidado')
    
    try:
        # Cargar tabla de ventas

        ##CONSOLIDADOS

        logging.info("Cargando tabla de ventas consolidado...")
        datos['tabla_de_ventas_consolidado'] = pd.read_excel(
            ruta_datos_consolidado + config['archivo_ventas_consolidado']
        )

        logging.info("Cargando tabla de consolidado costos...")
        datos['tabla_consolidado_costos'] = pd.read_excel(
            ruta_datos_consolidado + config['archivo_costos_consolidado']
        )

        logging.info("Cargando tabla de Sell out...")
        datos['tabla_sell_out_consolidado'] = pd.read_excel(
            ruta_datos_consolidado + config['archivo_tabla_sell_out']
        )

        logging.info("Cargando tabla de Sell out ciudades ...")
        datos['tabla_sell_out_ciudades_consolidado'] = pd.read_excel(
            ruta_datos_consolidado + config['archivo_tabla_sell_out_ciudades']
        )

        ## ENTRADA

        logging.info("Cargando tabla de ventas...")
        datos['tabla_ventas'] = pd.read_excel(
            ruta_datos + config['archivo_ventas'], 
            dtype=str
        )
        
        # Cargar tablas que no se actualizan frecuentemente
        logging.info("Cargando tabla de clientes...")
        datos['tabla_clientes'] = pd.read_excel(
            ruta_datos + config['archivo_clientes']
        )
        
        logging.info("Cargando tabla de cumplimiento...")
        datos['tabla_de_cumplimiento'] = pd.read_excel(
            ruta_datos + config['archivo_metas']
        )
        
        logging.info("Cargando tabla de costos...")
        datos['tabla_de_costos'] = pd.read_excel(
            ruta_datos + config['archivo_costos']
        )

        logging.info("Cargando tabla de marca referencia...")
        datos['tabla_mercado_referencia'] = pd.read_excel(
            ruta_datos + config['archivo_marca_referencia']
        )

        ### clientes grandes

        logging.info("Cargando tabla promotora...")
        datos['tabla_cliente_promotora'] = pd.read_excel(
            ruta_datos + config['archivo_promotora']
        )

        logging.info("Cargando tabla MAKRO...")
        datos['tabla_cliente_makro'] = pd.read_excel(
            ruta_datos + config['archivo_makro'],
            skiprows=18, header=[0, 1]
        )

        logging.info("Cargando tabla FARMATODO...")
        datos['tabla_cliente_farmatodo'] = pd.read_excel(
            ruta_datos + config['archivo_farmatodo']
        )
        
        
        logging.info("Todas las tablas cargadas exitosamente")
        
    except FileNotFoundError as e:
        logging.error(f"Archivo no encontrado: {e}")
        raise
    except Exception as e:
        logging.error(f"Error al cargar datos: {e}")
        raise
    
    return datos

def limpiar_formato_numerico_excel(serie):
    """
    Limpia formato numérico específico de Excel que puede causar problemas
    como separadores de miles, espacios, y otros caracteres de formato.
    
    Args:
        serie (pd.Series): Serie con valores potencialmente mal formateados
        
    Returns:
        pd.Series: Serie con formato numérico limpio
    """
    if serie.dtype == 'object':
        # Crear copia para no modificar original
        serie_limpia = serie.copy()
        
        # Limpiar solo valores no nulos
        mask_no_nulos = serie_limpia.notna()
        
        if mask_no_nulos.any():
            # Convertir a string para limpieza
            valores_str = serie_limpia[mask_no_nulos].astype(str)
            
            # Remover espacios
            valores_str = valores_str.str.replace(' ', '')
            
            # Remover separadores de miles comunes
            valores_str = valores_str.str.replace(',', '')
            valores_str = valores_str.str.replace('.', '', regex=False)  # Solo si es separador de miles
            
            # Detectar si hay demasiados ceros (posible error de formato)
            # Esto detecta números como 472800000 que deberían ser 4728
            def corregir_ceros_extra(valor_str):
                if pd.isna(valor_str) or valor_str == 'nan':
                    return valor_str
                
                # Si el string termina en muchos ceros, posiblemente está mal formateado
                if len(valor_str) > 4 and valor_str.endswith('00000'):
                    # Contar ceros al final
                    ceros_finales = 0
                    for i in range(len(valor_str)-1, -1, -1):
                        if valor_str[i] == '0':
                            ceros_finales += 1
                        else:
                            break
                    
                    # Si tiene 5 o más ceros al final, probablemente está mal
                    if ceros_finales >= 5:
                        # Remover los ceros extra (dejar máximo 2 para decimales)
                        valor_sin_ceros = valor_str.rstrip('0')
                        if len(valor_sin_ceros) > 0:
                            return valor_sin_ceros
                
                return valor_str
            
            # Aplicar corrección de ceros
            valores_str = valores_str.apply(corregir_ceros_extra)
            
            # Actualizar solo los valores que no eran nulos
            serie_limpia[mask_no_nulos] = valores_str
        
        return serie_limpia
    
    return serie


def limpiar_dataframe_excel(df):
    """
    Función para limpiar un DataFrame que proviene de un archivo Excel,
    eliminando formato y dejando solo los datos limpios.
    
    Args:
        df (pd.DataFrame): DataFrame original del Excel
        
    Returns:
        pd.DataFrame: DataFrame limpio
    """
    # Crear una copia para no modificar el original
    df_limpio = df.copy()
    
    # 1. Eliminar filas completamente vacías
    df_limpio = df_limpio.dropna(how='all')
    
    # 2. Eliminar columnas completamente vacías
    df_limpio = df_limpio.dropna(axis=1, how='all')
    
    # 3. Limpiar nombres de columnas (más conservador)
    if df_limpio.columns.dtype == 'object':
        # Quitar espacios en blanco al inicio y final
        df_limpio.columns = df_limpio.columns.str.strip()
        # Reemplazar espacios múltiples por uno solo
        df_limpio.columns = df_limpio.columns.str.replace(r'\s+', ' ', regex=True)
        # Solo quitar caracteres realmente problemáticos (no puntos ni guiones)
        df_limpio.columns = df_limpio.columns.str.replace(r'[^\w\s\.\-_]', '', regex=True)
    
    # 4. Limpiar datos en cada columna
    for columna in df_limpio.columns:
        if df_limpio[columna].dtype == 'object':
            # Convertir a string y limpiar espacios
            df_limpio[columna] = df_limpio[columna].astype(str).str.strip()
            
            # Reemplazar 'nan', 'None', cadenas vacías por NaN
            df_limpio[columna] = df_limpio[columna].replace(['nan', 'None', '', 'NaN'], np.nan)
            
            # Limpiar espacios múltiples
            df_limpio[columna] = df_limpio[columna].str.replace(r'\s+', ' ', regex=True)
            
            # Limpiar formato numérico de Excel antes de convertir
            df_limpio[columna] = limpiar_formato_numerico_excel(df_limpio[columna])
            
            # Intentar convertir a numérico si es posible (sin warnings)
            try:
                df_limpio[columna] = pd.to_numeric(df_limpio[columna])
            except (ValueError, TypeError):
                pass  # Mantener como texto si no se puede convertir
    
    # 5. Resetear índice
    df_limpio = df_limpio.reset_index(drop=True)
    
    # 6. Información del proceso de limpieza
    filas_eliminadas = len(df) - len(df_limpio)
    columnas_eliminadas = len(df.columns) - len(df_limpio.columns)
    
    print(f"Limpieza completada:")
    print(f"- Filas eliminadas: {filas_eliminadas}")
    print(f"- Columnas eliminadas: {columnas_eliminadas}")
    print(f"- Dimensiones finales: {df_limpio.shape}")
    
    return df_limpio


##PROCESAMIENTO VENTAS
def procesar_ventas(tabla_ventas, ultima_fecha, path):
    """
    Procesa la tabla de ventas aplicando todas las transformaciones necesarias.
    
    Args:
        tabla_ventas (pd.DataFrame): Tabla de ventas sin procesar
        ultima_fecha (pd.Timestamp): Fecha límite para filtrar ventas
        path (str): Ruta base para guardar archivos
        
    Returns:
        pd.DataFrame: Tabla de ventas procesada
    """
    logging.info("Iniciando procesamiento de ventas...")
    
    # Limpiar y convertir fechas
    tabla_ventas["Fecha"] = tabla_ventas["Fecha"].apply(lambda x: x.strip() if isinstance(x, str) else x)
    tabla_ventas["Fecha"] = pd.to_datetime(tabla_ventas["Fecha"], errors="coerce")

    
    # Filtrar fechas
    registros_antes = len(tabla_ventas)
    tabla_ventas = tabla_ventas[tabla_ventas["Fecha"] > ultima_fecha]
    registros_despues = len(tabla_ventas)
    logging.info(f"Filtrado por fecha: {registros_antes} -> {registros_despues} registros")
    
    # Renombrar columnas usando configuración
    tabla_ventas.rename(columns={"Desc. item": "Desc.item"}, inplace=True)
    
    # Agregar columna de tipo de identificación
    tabla_ventas["Tipo de identificacion"] = np.nan
    
    # Reordenar columnas
    cols = list(tabla_ventas.columns)
    cols.insert(3, cols.pop(cols.index("Tipo de identificacion")))
    tabla_ventas = tabla_ventas[cols]
    
    # Seleccionar solo las primeras 12 columnas
    tabla_ventas = tabla_ventas.iloc[:, :12]
    
    # Aplicar funciones de procesamiento
    tabla_ventas["Marca"] = tabla_ventas.apply(
        lambda row: funciones.asignar_marca(row["Desc.item"], row.get("Marca", np.nan)), 
        axis=1
    )
    
    tabla_ventas["TieneX"] = tabla_ventas["Desc.item"].apply(funciones.tiene_x)
    tabla_ventas["Cantidades"] = tabla_ventas["Desc.item"].apply(funciones.extraer_cantidad)
    tabla_ventas["Producto"] = tabla_ventas["Desc.item"].apply(funciones.extraer_producto)
    
    # Eliminar columna auxiliar
    tabla_ventas.drop(columns=["TieneX"], inplace=True)
    
    # Normalizar productos
    tabla_ventas["Producto_normalizado"] = tabla_ventas["Producto"].apply(funciones.normalizar_producto)
    tabla_ventas.drop(columns=["Producto"], inplace=True)
    tabla_ventas.rename(columns={"Producto_normalizado": "Producto"}, inplace=True)
    
    # Normalizar cantidades
    tabla_ventas["Cantidades"] = tabla_ventas["Cantidades"].apply(funciones.normalizar_cantidad)
    tabla_ventas["Cantidades"] = tabla_ventas.apply(
        lambda row: "1UND" if row["Cantidades"] == "NA" and not pd.isna(row["Desc.item"]) else row["Cantidades"], 
        axis=1
    )
    
    # Calcular cantidad sell int
    tabla_ventas["Cantidad inv.sell_Int"] = tabla_ventas.apply(funciones.calcular_cantidad_sell_int, axis=1)
    
    # Guardar tabla procesada
    archivo_salida = path + ARCHIVOS_SALIDA['VENTAS_PROCESADAS']
    tabla_ventas.to_excel(archivo_salida, index=False)
    logging.info(f"Tabla de ventas guardada en: {archivo_salida}")
    
    return tabla_ventas
    
def filtrar_registros_nuevos_ventas(df_entrada, df_consolidado):
    """
    Filtra los registros nuevos del archivo de entrada comparando con el consolidado histórico.
    """
    print("Verificando y estandarizando formatos de fecha...")

    campos_clave = [
        'Marca', 
        'Linea de mercado', 
        'Fecha', 
        'Referencia', 
        'Cantidad inv.',
        'Valor bruto local'
    ]

    # Convertir fechas a datetime para comparación
    df_entrada['Fecha'] = pd.to_datetime(df_entrada['Fecha'])
    df_consolidado['Fecha'] = pd.to_datetime(df_consolidado['Fecha'])
    
    # Obtener el último registro normalizado del consolidado
    ultimo_registro_normalizado = df_consolidado[campos_clave].astype(str)
    ultimo_registro_normalizado = limpiar_dataframe_excel(ultimo_registro_normalizado).iloc[-1]

    # 1. Encontrar fecha máxima del consolidado
    fecha_max_consolidado = df_consolidado['Fecha'].max()
    print(f"Fecha máxima en consolidado: {fecha_max_consolidado}")

    # 2. Filtrar entrada por fecha >= fecha máxima
    df_filtrado_fecha = df_entrada[df_entrada['Fecha'] >= fecha_max_consolidado].copy()
    print(f"Registros después del filtro por fecha: {len(df_filtrado_fecha)}")

    # Resetear índice para asegurar alineación
    df_filtrado_fecha = df_filtrado_fecha.reset_index(drop=True)

    # 3. Normalizar y limpiar datos
    entrada_normalizada = df_filtrado_fecha[campos_clave].astype(str)
    entrada_normalizada = limpiar_dataframe_excel(entrada_normalizada).reset_index(drop=True)

    print(f"Rango de fechas en entrada: {df_entrada['Fecha'].min()} a {df_entrada['Fecha'].max()}")

    # 4. Buscar coincidencias exactas
    coincidencias = entrada_normalizada.eq(ultimo_registro_normalizado).all(axis=1)
    print("Coincidencias exactas encontradas:", coincidencias.sum())

    """ # 5. Comparar columnas diferentes (opcional, útil para debug)
    if not coincidencias.any():
        diferencias = entrada_normalizada != ultimo_registro_normalizado
        columnas_diferentes = diferencias.any(axis=0)
        print("Columnas con diferencias respecto al último registro:")
        print(columnas_diferentes[columnas_diferentes].index.tolist())"""

    print("ULTIMO REGISTRO")
    print(ultimo_registro_normalizado)

    # 6. Tomar los registros nuevos correctamente después del match
    if coincidencias.any():
        idx = coincidencias[coincidencias].index[0]
        df_nuevos = df_filtrado_fecha.loc[idx+1:].copy()
        print(f"Se encontró el último registro del consolidado en la entrada. Nuevos desde el índice {idx + 1}.")
    else:
        df_nuevos = df_filtrado_fecha.copy()
        print("El último registro del consolidado NO se encontró en la entrada. Se toman todos como nuevos.")

    print(f"Registros nuevos encontrados: {len(df_nuevos)}")
    return df_nuevos

## Procesamiento de clientes 
def procesar_clientes(tabla_clientes):
    """
    Procesa la tabla de clientes eliminando duplicados.
    
    Args:
        tabla_clientes (pd.DataFrame): Tabla de clientes sin procesar
        
    Returns:
        pd.DataFrame: Tabla de clientes procesada
    """
    # Eliminar duplicados por número de identificación
    tabla_clientes = tabla_clientes.drop_duplicates(subset=["nit"])
    
    return tabla_clientes


## procesar costos
def procesar_costos_consolidado(tabla_costos, tabla_marca_referencia):
    """
    Procesa y consolida las tablas de costos.
    
    Returns:
        pd.DataFrame: Tabla de costos consolidada
    """
    # Fusionar las tablas
    consolidado_costo = tabla_costos.merge(tabla_marca_referencia, on="Referencia", how="left")
    
    # Guardar el archivo consolidado
    consolidado_costo.to_excel("Consolidado CostosINICIO.xlsx", index=False)
    
    return consolidado_costo
    

##conteo mercado
def crear_conteo_linea_mercado(tabla_ventas):
    """
    Crea el conteo de compras por línea de mercado agrupado por semana, año y mes.
    
    Args:
        tabla_ventas (pd.DataFrame): Tabla de ventas procesada
        
    Returns:
        pd.DataFrame: Conteo de compras por línea de mercado
    """
    # Diccionario de conversión de meses
    month_mapping = {
    # Inglés
    "january": "01", "february": "02", "march": "03", "april": "04",
    "may": "05", "june": "06", "july": "07", "august": "08",
    "september": "09", "october": "10", "november": "11", "december": "12",
    
    # Español
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
    "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
    "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
}

    
    # Asegurar que la columna Fecha es tipo datetime
    tabla_ventas["Fecha"] = pd.to_datetime(tabla_ventas["Fecha"], errors='coerce')
    
    # Crear columna de semana
    tabla_ventas["Semana"] = tabla_ventas["Fecha"].dt.to_period("W").apply(lambda r: r.start_time)
    
    # Extraer mes y año
    tabla_ventas["Mes"] = tabla_ventas["Semana"].dt.strftime("%B").str.lower()
    tabla_ventas["Año"] = tabla_ventas["Semana"].dt.year
    tabla_ventas["Mes_num"] = tabla_ventas["Mes"].map(month_mapping)

    print(tabla_ventas.head())
    
    # Agrupar y contar compras
    ComprasSemanaAñoMes = tabla_ventas.groupby([
        "Año", "Mes", "Semana", "Linea de mercado", "Mes_num", "Razon social"
    ]).size().reset_index(name="numCompras")
    
    # Guardar en Excel
    ComprasSemanaAñoMes.to_excel("Conteo linea de mercado.xlsx", index=False)
    
    return ComprasSemanaAñoMes


def crear_tabla_marca_referencia(Tabla_marca_referencia_y_producto, tabla_ventas):
    """
    Crea la tabla de marca, referencia y producto.
    
    Returns:
        pd.DataFrame: Tabla de marca, referencia y producto
    """

    print(Tabla_marca_referencia_y_producto.columns)
    print(tabla_ventas.columns)

    
    # Fusionar tablas
    Tabla_marca_referencia_y_producto = tabla_ventas.merge(
        Tabla_marca_referencia_y_producto, on="Referencia", how="outer"
    )

    # Revisar columnas después del merge
    print(Tabla_marca_referencia_y_producto.columns)

    # Elegir cuál EAN conservar y renombrar
    Tabla_marca_referencia_y_producto["EAN"] = Tabla_marca_referencia_y_producto["EAN 13_x"].combine_first(
        Tabla_marca_referencia_y_producto["EAN 13_y"]
    )

    # Luego eliminar las columnas originales
    Tabla_marca_referencia_y_producto = Tabla_marca_referencia_y_producto.drop(columns=["EAN 13_x", "EAN 13_y"])



    # Renombrar columnas
    Tabla_marca_referencia_y_producto = Tabla_marca_referencia_y_producto.rename(
        columns={"Desc.item": "Desc._item.Sell.Out"}
    )
    
    # Seleccionar columnas necesarias
    Tabla_marca_referencia_y_producto = Tabla_marca_referencia_y_producto[
        ["MARCA", "Referencia", "Desc._item", "EAN", "Desc._item.Sell.Out"]
    ]
    
    # Mapeo de marcas
    marca_mapping = {
        "NAT": "001 - NATURESSE",
        "JDE": "002 - JARDIN DE EVA",
        "CED": "003 - CEDRELA",
        "AMA": "004 - AMATISTA",
        "INM": "006 - INMARCESIBLE",
        "MED": "007 - MEDELA",
        "AWA": "008 - AWARA",
        "PER": "010 - PERSONALIZADO"
    }
    
    def asignar_marca(desc):
        if pd.isna(desc):
            return np.nan
        for key, value in marca_mapping.items():
            if desc.startswith(key):
                return value
        return np.nan
    
    # Aplicar asignación de marca
    Tabla_marca_referencia_y_producto["MARCA"] = Tabla_marca_referencia_y_producto["Desc._item"].apply(asignar_marca)
    
    # Completar valores faltantes
    Tabla_marca_referencia_y_producto["Desc._item.Sell.Out"] = Tabla_marca_referencia_y_producto.apply(
        lambda row: row["Desc._item"] if pd.isna(row["Desc._item.Sell.Out"]) else row["Desc._item.Sell.Out"], 
        axis=1
    )
    
    # Filtrar y limpiar
    Tabla_marca_referencia_y_producto = Tabla_marca_referencia_y_producto.dropna(subset=["MARCA"])
    Tabla_marca_referencia_y_producto = Tabla_marca_referencia_y_producto.drop_duplicates(subset=["MARCA", "EAN"])
    
    # Guardar
    Tabla_marca_referencia_y_producto.to_excel("Tabla marca referencia y producto2.xlsx", index=False)
    
    return Tabla_marca_referencia_y_producto


def consolidar_costos_historicos(tabla_costos_consolidado, tabla_costo_entrada, tabla_marca_referencia):
    """
    Consolida los costos históricos con los datos del año actual.
    
    Returns:
        pd.DataFrame: Costos consolidados
    """
    
    # Procesar tabla de costos 2024
    tabla_costo_entrada = tabla_costo_entrada.rename(
        columns={"Costo prom. uni.": "Costo prom.", "Desc. item": "Desc.item"}
    )[["Marca", "Referencia", "Desc.item", "Instalación", "U.M.", "Costo prom.", "Mes", "Año"]]
    
    # Fusionar con referencias
    tabla_costo_entrada = tabla_costo_entrada.merge(tabla_marca_referencia, on="Referencia", how="left")
    
    # Procesar tabla histórica
    consolidado_costo = tabla_costos_consolidado.merge(tabla_marca_referencia, on="Referencia", how="left")

    print(consolidado_costo)
    
    # Luego filtras columnas
    consolidado_costo1 = consolidado_costo[["Referencia", "Desc.item", "Instalación", "U.M.", "Costo prom.", "Mes", "Año", "Marca", "Incluir referencia(Si/No)"]]

    """consolidado_costo1 = consolidado_costo.rename(
        columns={"Incluir referencia(Si/No).y": "Incluir referencia(Si/No)"}
    )[["Referencia", "Desc.item", "Instalación", "U.M.", "Costo prom.", "Mes", "Año", "Marca", "Incluir referencia(Si/No)"]]"""
    
    # Filtrar años anteriores
    max_year = consolidado_costo1["Año"].max()
    consolidado_costo1 = consolidado_costo1[consolidado_costo1["Año"] < max_year]

    print(consolidado_costo1.columns)
    
    # Unir datos
    consolidado_de_costo = pd.concat([consolidado_costo1, tabla_costo_entrada], ignore_index=True)
    
    # Guardar
    consolidado_de_costo.to_excel("Consolidado Costos.xlsx", index=False)
    
    return consolidado_de_costo



In [104]:
 # 1. CONFIGURACIÓN INICIAL
print("📁 Configurando rutas y parámetros...")
config = configurar_rutas()
print(f"✅ Rutas configuradas: {config['path']}")

 # 2. CARGA DE DATOS
print("\n📊 Cargando datos desde archivos Excel...")
datos = cargar_datos(config)
print(f"✅ Tablas cargadas:")
for nombre, tabla in datos.items():
    print(f"   - {nombre}: {tabla.shape[0]} filas, {tabla.shape[1]} columnas")

📁 Configurando rutas y parámetros...
✅ Rutas configuradas: C:/Users/Usuario/Downloads/De r a python/

📊 Cargando datos desde archivos Excel...


C:\Users\Usuario\Downloads\De r a python\codigos\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell D2 is marked as a date but the serial value 71704739 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
C:\Users\Usuario\Downloads\De r a python\codigos\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell D3 is marked as a date but the serial value 71704739 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
C:\Users\Usuario\Downloads\De r a python\codigos\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell D4 is marked as a date but the serial value 66658043 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
C:\Users\Usuario\Downloads\De r a python\codigos\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell D5 is marked as a date but the serial value 71704739 is outside the limits for dates. The ce

✅ Tablas cargadas:
   - tabla_de_ventas_consolidado: 12571 filas, 15 columnas
   - tabla_consolidado_costos: 6684 filas, 9 columnas
   - tabla_sell_out_consolidado: 72 filas, 5 columnas
   - tabla_sell_out_ciudades_consolidado: 16114 filas, 9 columnas
   - tabla_ventas: 12677 filas, 11 columnas
   - tabla_clientes: 2893 filas, 3 columnas
   - tabla_de_cumplimiento: 96 filas, 6 columnas
   - tabla_de_costos: 5241 filas, 9 columnas
   - tabla_mercado_referencia: 309 filas, 6 columnas
   - tabla_cliente_promotora: 485 filas, 14 columnas
   - tabla_cliente_makro: 132 filas, 34 columnas
   - tabla_cliente_farmatodo: 462 filas, 13 columnas


## PROCESAMIENTO VENTAS E HISTORICO

In [47]:
df_nuevos = filtrar_registros_nuevos_ventas(datos['tabla_ventas'], datos['tabla_de_ventas_consolidado'])

# 3. PROCESAMIENTO DE VENTAS
print("\n🔄 Procesando tabla de ventas...")
tabla_ventas_procesada = procesar_ventas(
    df_nuevos, 
    config['ultima_fecha'], 
    config['path']
)

print(tabla_ventas_procesada)
print(f"✅ Ventas procesadas: {tabla_ventas_procesada.shape[0]} registros")

# 4. CONCATENAR Y GUARDAR LISTADO HISTÓRICO ACTUALIZADO
print("\n📦 Concatenando con consolidado histórico...")

# Asegúrate de tener el consolidado cargado
df_consolidado = datos['tabla_de_ventas_consolidado']

# Concatenar nuevos datos con el consolidado
df_actualizado = pd.concat([df_consolidado, tabla_ventas_procesada], ignore_index=True)

# Guardar como archivo Excel
ruta_salida = os.path.join(config['path'], 'listado_historico_ventas_actualizado.xlsx')
df_actualizado.to_excel(ruta_salida, index=False)

print(f"✅ Archivo actualizado guardado en: {ruta_salida}")


Verificando y estandarizando formatos de fecha...
Limpieza completada:
- Filas eliminadas: 0
- Columnas eliminadas: 0
- Dimensiones finales: (12571, 6)
Fecha máxima en consolidado: 2025-05-22 00:00:00
Registros después del filtro por fecha: 109
Limpieza completada:
- Filas eliminadas: 0
- Columnas eliminadas: 0
- Dimensiones finales: (109, 6)
Rango de fechas en entrada: 2023-01-02 00:00:00 a 2025-05-27 00:00:00
Coincidencias exactas encontradas: 1
ULTIMO REGISTRO
Marca                  001-NATURESSE
Linea de mercado     014-EXPORTACION
Fecha                     2025-05-22
Referencia                     10002
Cantidad inv.                   1017
Valor bruto local            5298570
Name: 12570, dtype: object
Se encontró el último registro del consolidado en la entrada. Nuevos desde el índice 3.
Registros nuevos encontrados: 106

🔄 Procesando tabla de ventas...
                   Marca     Linea de mercado                Nombre vendedor  \
3        001 - NATURESSE    014 - EXPORTACION  S

## PROCESAMIENTO DE CLIENTES

In [48]:
# PROCESAMIENTO DE clientes
print("\n🔄 Procesando tabla de clientes...")
tabla_clientes_procesada = procesar_clientes(
    datos['tabla_clientes']
)
print(f"✅ clientes procesados: {tabla_clientes_procesada.shape[0]} registros")

tabla_clientes_procesada.to_excel("tabla_clientes.xlsx", index=False)
print("💾 Archivo 'tabla_clientes.xlsx' guardado correctamente.")


🔄 Procesando tabla de clientes...
✅ clientes procesados: 2830 registros
💾 Archivo 'tabla_clientes.xlsx' guardado correctamente.


## PROCESAMINETO COSTOS CON TABLA MERCADO Y REFERENCIA (PENSAR Y PREGUNTAR EL PORQUE ESA NO SE USA

In [49]:
# PROCESAMIENTO DE clientes
print("\n🔄 Procesar el consolidado de costos...")
costos_consolidado = procesar_costos_consolidado(
    datos['tabla_consolidado_costos'],
    datos['tabla_mercado_referencia']
)
print(f"✅ tabla costos_consolidado: {costos_consolidado.shape[0]} registros")


🔄 Procesar el consolidado de costos...
✅ tabla costos_consolidado: 6716 registros


## TABLA CONTEO MERCADO

In [50]:
# PROCESAMIENTO DE clientes
print("\n🔄 Creando tabla de conteo mercado...")
tabla_conteo_mercado = crear_conteo_linea_mercado(
    df_actualizado
)
print(f"✅ tabla conteo mercado: {tabla_conteo_mercado.shape[0]} registros")


🔄 Creando tabla de conteo mercado...
                 Marca Linea de mercado            Nombre vendedor  \
0  002 - JARDIN DE EVA    003 - WEBSITE  MERCADOPAGO COLOMBIA LTDA   
1  002 - JARDIN DE EVA    003 - WEBSITE  MERCADOPAGO COLOMBIA LTDA   
2  002 - JARDIN DE EVA    003 - WEBSITE  MERCADOPAGO COLOMBIA LTDA   
3  002 - JARDIN DE EVA    003 - WEBSITE  MERCADOPAGO COLOMBIA LTDA   
4      001 - NATURESSE    003 - WEBSITE  MERCADOPAGO COLOMBIA LTDA   

   Tipo de identificacion       NIT                   Razon social Referencia  \
0                     NaN  71704739        RESTREPO JUAN CRISTOBAL      10064   
1                     NaN  71704739        RESTREPO JUAN CRISTOBAL      10066   
2                     NaN  66658043         CUELLAR LATORRE ELIANA      10067   
3                     NaN  71704739        RESTREPO JUAN CRISTOBAL      10065   
4                     NaN  52933684  SUAREZ PINTO ANDREA DEL PILAR      10034   

                       Desc.item      Fecha Cantidad i

## CREAR TABLA MARCA Y REFERENCIA NUEVA - preguntar

In [54]:
# PROCESAMIENTO DE clientes
print("\n🔄 Creando tabla de marca y referencia nueva...")
tabla_marca_referencia = crear_tabla_marca_referencia(
    datos['tabla_mercado_referencia'], 
    df_actualizado
)
print(f"✅ tabla de marca y referencia: {tabla_marca_referencia.shape[0]} registros")


🔄 Creando tabla de marca y referencia nueva...
Index(['MARCA', 'Referencia', 'Desc._item', 'Incluir referencia(Si/No)',
       'EAN 13', 'tipo producto'],
      dtype='object')
Index(['Marca', 'Linea de mercado', 'Nombre vendedor',
       'Tipo de identificacion', 'NIT', 'Razon social', 'Referencia',
       'Desc.item', 'Fecha', 'Cantidad inv.', 'Valor bruto local', 'EAN',
       'Cantidades', 'Producto', 'Cantidad inv.sell_Int', 'EAN 13', 'Semana',
       'Mes', 'Año', 'Mes_num'],
      dtype='object')
Index(['Marca', 'Linea de mercado', 'Nombre vendedor',
       'Tipo de identificacion', 'NIT', 'Razon social', 'Referencia',
       'Desc.item', 'Fecha', 'Cantidad inv.', 'Valor bruto local', 'EAN',
       'Cantidades', 'Producto', 'Cantidad inv.sell_Int', 'EAN 13_x', 'Semana',
       'Mes', 'Año', 'Mes_num', 'MARCA', 'Desc._item',
       'Incluir referencia(Si/No)', 'EAN 13_y', 'tipo producto'],
      dtype='object')
✅ tabla de marca y referencia: 83 registros


## CONSOLIDAR COSTOS HISTORICOS

In [55]:
# PROCESAMIENTO DE clientes
print("\n🔄 Creando tabla de marca y referencia nueva...")
consolidado_costos_historico = consolidar_costos_historicos(
    datos['tabla_consolidado_costos'], 
    datos['tabla_de_costos'], 
    tabla_marca_referencia
)
print(f"✅ tabla costos historicos: {consolidado_costos_historico.shape[0]} registros")


🔄 Creando tabla de marca y referencia nueva...
     Referencia                  Desc.item  Instalación    U.M.  Costo prom.  \
0           103         KIT DE AFEITAR GPS          1.0     UND       1145.0   
1           103         KIT DE AFEITAR GPS          1.0     UND       1145.0   
2           103         KIT DE AFEITAR GPS          1.0     UND       1145.0   
3           103         KIT DE AFEITAR GPS          1.0     UND       1145.0   
4           103         KIT DE AFEITAR GPS          1.0     UND       1145.0   
...         ...                        ...          ...     ...          ...   
6679         99  PAÑUELO FACIAL CASA ONIRI          1.0  UNIDAD        112.0   
6680         99  PAÑUELO FACIAL CASA ONIRI          1.0  UNIDAD        112.0   
6681         99  PAÑUELO FACIAL CASA ONIRI          1.0  UNIDAD        112.0   
6682         99  PAÑUELO FACIAL CASA ONIRI          1.0  UNIDAD        112.0   
6683         99  PAÑUELO FACIAL CASA ONIRI          1.0  UNIDAD        1

# PROCESAMIENTO CLIENTES GRANDES

## Procesamiento promotora

In [134]:
import pandas as pd
from datetime import datetime

# Diccionario de conversión de meses
meses_mapping = {
    "ENE": "1", "FEB": "2", "MAR": "3", "ABR": "4",
    "MAY": "5", "JUN": "6", "JUL": "7", "AGO": "8",
    "SEP": "9", "OCT": "10", "NOV": "11", "DIC": "12"
}

clientes_mapping = {
    "PROMOTORA DE COMERCIO SOCIAL",
    "FARMATODO COLOMBIA SA",
    "MAKRO SUPERMAYORISTA SAS"
}

def obtener_numero_mes(nombre_mes):
    return int(meses_mapping[nombre_mes])
    
def obtener_siguiente_mes(mes_actual, año_actual):
    """
    Calcula el siguiente mes dado un mes y año actual
    
    Args:
        mes_actual: Mes en formato texto (ej: "AGO")
        año_actual: Año en formato numérico
    
    Returns:
        tuple: (siguiente_mes, siguiente_año)
    """
    # Mapeo inverso para convertir número a mes
    numero_a_mes = {
        1: "ENE", 2: "FEB", 3: "MAR", 4: "ABR",
        5: "MAY", 6: "JUN", 7: "JUL", 8: "AGO",
        9: "SEP", 10: "OCT", 11: "NOV", 12: "DIC"
    }
    
    # Obtener número del mes actual
    num_mes_actual = int(meses_mapping[mes_actual])
    
    # Calcular siguiente mes
    if num_mes_actual == 12:  # Si es diciembre
        siguiente_mes_num = 1
        siguiente_año = año_actual + 1
    else:
        siguiente_mes_num = num_mes_actual + 1
        siguiente_año = año_actual
    
    siguiente_mes = numero_a_mes[siguiente_mes_num]
    
    return siguiente_mes, siguiente_año

def obtener_ultimo_mes_cliente(consolidado_sell_out, nombre_cliente):
    """
    Encuentra el último mes registrado para un cliente específico
    
    Args:
        archivo_excel: Ruta del archivo Excel
        nombre_cliente: Nombre del cliente a buscar
    
    Returns:
        dict: Información del último registro del cliente
    """
    try:
        
        # Filtrar por el cliente específico
        df_cliente = consolidado_sell_out[consolidado_sell_out['cliente'] == nombre_cliente].copy()
        
        if df_cliente.empty:
            return {"error": f"No se encontraron registros para el cliente: {nombre_cliente}"}
        
        # Crear una columna de fecha para ordenar cronológicamente
        df_cliente['fecha_ordenamiento'] = pd.to_datetime(
            df_cliente['año'].astype(str) + '-' + 
            df_cliente['Mes'].map(meses_mapping) + '-01'
        )
        
        # Ordenar por fecha y obtener el último registro
        df_cliente_ordenado = df_cliente.sort_values('fecha_ordenamiento')
        ultimo_registro = df_cliente_ordenado.iloc[-1]
        
        return {
            "cliente": ultimo_registro['cliente'],
            "ultimo_mes": ultimo_registro['Mes'],
            "ultimo_año": ultimo_registro['año'],
            "fecha_completa": f"{ultimo_registro['Mes']} {ultimo_registro['año']}",
            "unidades": ultimo_registro['Unidades'],
            "total_registros": len(df_cliente),
            "siguiente_mes": obtener_siguiente_mes( ultimo_registro['Mes'], ultimo_registro['año'])
        }
        
    except FileNotFoundError:
        return {"error": "Archivo no encontrado"}
    except Exception as e:
        return {"error": f"Error al procesar el archivo: {str(e)}"}

def separar_tienda_ciudad(nombre):
    nombre = nombre.strip()
    
    # Casos especiales
    if nombre == "CARULLA PRADERA DE POTOSI":
        return "CARULLA PRADERA DE POTOSI", "POTOSI"
    elif nombre == "CARULLA MANIZALES":
        return "CARULLA MANIZALES", "MANIZALES"
    elif nombre == "CARULLA PLAZA CLARO":
        return "CARULLA PLAZA CLARO", "BOGOTA"
    
    # Caso general (separado por guion '-')
    if '-' in nombre:
        partes = [x.strip() for x in nombre.split('-')]
        tienda = partes[0]
        ciudad = partes[1]
        return tienda, ciudad
    
    # Fallback por si hay algo inesperado
    return nombre, "CIUDAD DESCONOCIDA"
        
def procesar_promotora(archivo_entrada_promotora):
    """
    Procesa los datos de ventas de PROMOTORA.
    
    Returns:
        pd.DataFrame: Ventas de PROMOTORA procesadas
    """
    
    informacion_consolidado= obtener_ultimo_mes_cliente(datos['tabla_sell_out_consolidado'], "PROMOTORA DE COMERCIO SOCIAL")

    if 'Unidades' not in archivo_entrada_promotora.columns:
        return {"error": "No se encontró la columna 'Unidades' en el archivo de promotora"}

    archivo_entrada_promotora["Fecha"] = pd.to_datetime(archivo_entrada_promotora["Fecha"], errors='coerce')
    archivo_entrada_promotora["Mes"] = archivo_entrada_promotora["Fecha"].dt.strftime("%B").str.lower()
    archivo_entrada_promotora["Año"] = archivo_entrada_promotora["Fecha"].dt.year

    archivo_entrada_promotora = archivo_entrada_promotora[
        archivo_entrada_promotora['Nomb. Dependencia'].notna() & 
        (archivo_entrada_promotora['Nomb. Dependencia'].str.strip() != '')
    ]
    
        
    archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(
    lambda x: pd.Series(separar_tienda_ciudad(x))
    )

    siguiente_mes, siguiente_año = informacion_consolidado['siguiente_mes']
    num_mes = obtener_numero_mes(siguiente_mes)

    
    ventas_promotora = archivo_entrada_promotora.groupby([
       "EAN", "Mes", "Año", "Tienda", "Ciudad", "Descripción" ]).agg({"Unidades": "sum"}).reset_index()

    ventas_promotora = ventas_promotora.rename(columns={"Año": "año", "Descripción":"Descripcion"})

    pos_ean = ventas_promotora.columns.get_loc("EAN")
    ventas_promotora.insert(pos_ean + 1, "cliente", "PROMOTORA DE COMERCIO SOCIAL")
    

    pos = ventas_promotora.columns.get_loc("Unidades")
    ventas_promotora.insert(pos + 1, "NumMes", num_mes)

    total_unidades = int(ventas_promotora["Unidades"].sum())

    consolidado_sell_out = {
        "Mes": siguiente_mes,
        "año": siguiente_año,
        "cliente": "PROMOTORA DE COMERCIO SOCIAL",
        "Unidades": total_unidades,
        "NumMes": num_mes
    }
    
    return consolidado_sell_out, ventas_promotora

In [64]:
consolidado_sell_out_promotora, ventas_promotora_ciudad_promotora = procesar_promotora(datos['tabla_cliente_promotora'])

consolidado_sell_out_promotora, ventas_promotora_ciudad_promotora

C:\Users\Usuario\AppData\Local\Temp\ipykernel_35956\1800045614.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35956\1800045614.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(


({'Mes': 'MAR',
  'año': np.int64(2025),
  'cliente': 'PROMOTORA DE COMERCIO SOCIAL',
  'Unidades': 533,
  'NumMes': 3},
           EAN                       cliente    Mes     año  \
 0    7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 1    7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 2    7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 3    7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 4    7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 ..        ...                           ...    ...     ...   
 241  7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 242  7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 243  7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 244  7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 245  7.71e+12  PROMOTORA DE COMERCIO SOCIAL  march  2025.0   
 
                         Tienda     Ciudad  \
 0    CARULLA PRADERA DE POTOSI     POTOSI   
 1       CARULLA QUINTA CAMAC

## Procesamiento Makro

In [135]:
meses_makro = {
    "ENE": "Enero", "FEB": "Febrero", "MAR": "Marzo", "ABR": "Abril",
    "MAY": "Mayo", "JUN": "Junio", "JUL": "Julio", "AGO": "Agosto",
    "SEP": "Septiembre", "OCT": "Octubre", "NOV": "Noviembre", "DIC": "Diciembre"
}

def obtener_mes_completo(nombre_mes):
    return meses_makro[nombre_mes]

def separar_tienda_ciudad_makro(valor):
    if pd.isna(valor):
        return pd.Series(["", ""])
    
    partes = valor.split("-", 1)
    nombre = partes[1].strip() if len(partes) == 2 else valor.strip()
    nombre = nombre.upper()

    if nombre == "MAKRO CALLE 30":
        return pd.Series(["MAKRO CALLE 30", "SOLEDAD"])
    elif nombre == "ESTACION POBLADO":
        return pd.Series(["ESTACION POBLADO", "MEDELLIN"])
    elif nombre == "CALI - NORTE":
        return pd.Series(["NORTE", "CALI"])
    
    if " - " in nombre:
        partes = nombre.rsplit(" - ", 1)
        if len(partes) == 2:
            tienda, ciudad = partes
            return pd.Series([tienda.strip(), ciudad.strip()])

    return pd.Series(["Tienda Única", nombre.strip()])

def procesar_makro(archivo_entrada_makro):
    """
    Procesa los datos de ventas de MAKRO.
    
    Returns:
        pd.DataFrame: Ventas de MAKRO procesadas
    """

    informacion_consolidado= obtener_ultimo_mes_cliente(datos['tabla_sell_out_consolidado'], "MAKRO SUPERMAYORISTA SAS")
    siguiente_mes, siguiente_año = informacion_consolidado['siguiente_mes']

    mes_procesar = obtener_mes_completo(siguiente_mes)
    
    id_vars = [
        ('Tienda', 'Unnamed: 6_level_1'),
        ('Número de artículo Makro', 'Unnamed: 2_level_1'),
        ('Descripción del artículo', 'Unnamed: 1_level_1'),
        ('Codigo de barras', 'Unnamed: 5_level_1')
    ]

    df_completo = archivo_entrada_makro.copy()
    for col in id_vars:
        if col in df_completo.columns:
            df_completo = df_completo[
                df_completo[col].notna() & 
                (df_completo[col] != '') & 
                (df_completo[col] != ' ')
            ]

    columna_mes_cde = [col for col in df_completo.columns if col == (mes_procesar, 'Cde.')]

    if not columna_mes_cde:
        print("No se encontraron columnas Cde_. Intentando buscar patrones alternativos...")
        columnas_cde = [col for col in df_completo.columns if 'Cde_' in str(col)]
        print(f"Columnas con 'Cde' encontradas: {columnas_cde}")
        
    MAKRO_consolidado_long = df_completo.melt(
        id_vars=id_vars, 
        value_vars=columna_mes_cde,
        var_name="Mes", 
        value_name="Unidades"
    )

    MAKRO_consolidado_long = MAKRO_consolidado_long.dropna(subset=['Unidades'])
    MAKRO_consolidado_long = MAKRO_consolidado_long[MAKRO_consolidado_long['Unidades'] != 0]  # Opcional: excluir ceros
    
    MAKRO_consolidado_long["Mes"] = (MAKRO_consolidado_long["Mes"]
                                   .str.replace("Cde.", "")
                                   .str.lower())

    MAKRO_consolidado_long["Año"] = siguiente_año
    MAKRO_consolidado_long["cliente"] = "MAKRO SUPERMAYORISTA SAS"
    

    col_tienda_raw = ('Tienda', 'Unnamed: 6_level_1')

    # Aplicar la función
    MAKRO_consolidado_long[['Tienda', 'Ciudad']] = MAKRO_consolidado_long[col_tienda_raw].apply(
        separar_tienda_ciudad_makro
    )

    Venta_Unidades_MAKRO = MAKRO_consolidado_long.groupby([
        ('Codigo de barras', 'Unnamed: 5_level_1'), "Mes", "Año", "cliente", "Tienda", "Ciudad", ('Descripción del artículo', 'Unnamed: 1_level_1')
    ]).agg({"Unidades": "sum"}).reset_index()

    total_unidades = int(MAKRO_consolidado_long["Unidades"].sum())
    num_mes = obtener_numero_mes(siguiente_mes)

    Venta_Unidades_MAKRO = Venta_Unidades_MAKRO.rename(columns={"Año": "año", ('Codigo de barras', 'Unnamed: 5_level_1'):"EAN", ('Descripción del artículo', 'Unnamed: 1_level_1'):"Descripcion"})

    pos = Venta_Unidades_MAKRO.columns.get_loc("Unidades")
    Venta_Unidades_MAKRO.insert(pos + 1, "NumMes", num_mes)


    consolidado_sell_out = {
        "Mes": siguiente_mes,
        "año": siguiente_año,
        "cliente": "MAKRO SUPERMAYORISTA SAS",
        "Unidades": int(MAKRO_consolidado_long["Unidades"].sum()),
        "NumMes": num_mes
    }
    
    return consolidado_sell_out, Venta_Unidades_MAKRO


In [98]:
procesar_makro(datos['tabla_cliente_makro'])

({'Mes': 'MAR',
  'año': np.int64(2025),
  'cliente': 'MAKRO SUPERMAYORISTA SAS',
  'Unidades': 2556,
  'NumMes': 3},
           EAN    Mes   año                   cliente            Tienda  \
 0    7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS    AVENIDA BOYACA   
 1    7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS            CUMARA   
 2    7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS     DOS QUEBRADAS   
 3    7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS  ESTACION POBLADO   
 4    7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS    MAKRO CALLE 30   
 ..        ...    ...   ...                       ...               ...   
 98   7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS      Tienda Única   
 99   7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS      Tienda Única   
 100  7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS    VALLE DEL LILI   
 101  7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS     VILLA DEL RIO   
 102  7.71e+12  marzo  2025  MAKRO SUPERMAYORISTA SAS    

## Procesamiento Farmatodo

In [136]:
def procesar_farmatodo(archivo_farmatodo):
    """
    Procesa los datos de ventas de FARMATODO.
    
    Returns:
        pd.DataFrame: Ventas de FARMATODO procesadas
    """

    informacion_consolidado= obtener_ultimo_mes_cliente(datos['tabla_sell_out_consolidado'], "FARMATODO COLOMBIA SA")
    siguiente_mes, siguiente_año = informacion_consolidado['siguiente_mes']
    num_mes = obtener_numero_mes(siguiente_mes)

    farmatodo_tabla = archivo_farmatodo.rename(columns={
    "UNIDADES_VENDIDAS": "Unidades", 
    "CIUDAD": "Ciudad", 
    "NOMBRE_TIENDA": "Tienda",
    "DESCRIPCION_ITEM": "Descripcion", 
    "UPC": "EAN", 
    "SUP_NAME": "SUP_NAME"
    })

    farmatodo_tabla["Mes"] = siguiente_mes
    farmatodo_tabla["año"] = siguiente_año
    farmatodo_tabla["cliente"] = "FARMATODO COLOMBIA SA"

    farmatodo_tabla = farmatodo_tabla.groupby([
        "EAN", "Mes", "año", "cliente", "Tienda", "Ciudad", "Descripcion"
    ]).agg({"Unidades": "sum"}).reset_index()

    pos = farmatodo_tabla.columns.get_loc("Unidades")
    farmatodo_tabla.insert(pos + 1, "NumMes", num_mes)

    consolidado_sell_out = {
        "Mes": siguiente_mes,
        "año": siguiente_año,
        "cliente": "FARMATODO COLOMBIA SA",
        "Unidades": int(archivo_farmatodo["UNIDADES_VENDIDAS"].sum()),
        "NumMes": num_mes
    }
    
    return consolidado_sell_out, farmatodo_tabla

In [125]:
procesar_farmatodo(datos['tabla_cliente_farmatodo'])

({'Mes': 'ABR',
  'año': np.int64(2025),
  'cliente': 'FARMATODO COLOMBIA SA',
  'Unidades': 846,
  'NumMes': 4},
                EAN  Mes   año                cliente             Tienda  \
 0    7708009232370  ABR  2025  FARMATODO COLOMBIA SA         Carrera 13   
 1    7708009232370  ABR  2025  FARMATODO COLOMBIA SA            Colores   
 2    7708009232370  ABR  2025  FARMATODO COLOMBIA SA          El Tesoro   
 3    7708009232370  ABR  2025  FARMATODO COLOMBIA SA        San Nicolas   
 4    7708009232370  ABR  2025  FARMATODO COLOMBIA SA  Viva Barranquilla   
 ..             ...  ...   ...                    ...                ...   
 383  7709967073999  ABR  2025  FARMATODO COLOMBIA SA            Usaquen   
 384  7709967073999  ABR  2025  FARMATODO COLOMBIA SA      Variante Chia   
 385  7709967073999  ABR  2025  FARMATODO COLOMBIA SA          Versalles   
 386  7709967073999  ABR  2025  FARMATODO COLOMBIA SA     Villa Carolina   
 387  7709967073999  ABR  2025  FARMATODO COLOMBIA

## TABLA SELL OUT CONSOLIDADO (FALTA LOS CASOS EN LOS QUE NO SE SUMINISTRE ALGUN EXCEL DE ESTOS)

In [126]:
import pandas as pd

def actualizar_consolidado_y_guardar_excel(datos, nombre_archivo_excel):
    
    consolidado_promotora, _ = procesar_promotora(datos['tabla_cliente_promotora'])
    df_promotora = pd.DataFrame([consolidado_promotora])

    consolidado_makro, _ = procesar_makro(datos['tabla_cliente_makro'])
    df_makro = pd.DataFrame([consolidado_makro])

    consolidado_farmatodo, _ = procesar_farmatodo(datos['tabla_cliente_farmatodo'])
    df_farmatodo = pd.DataFrame([consolidado_farmatodo])

    df_sell_out_actualizado = pd.concat([
        datos['tabla_sell_out_consolidado'],
        df_promotora,
        df_makro,
        df_farmatodo
    ], ignore_index=True)

    df_sell_out_actualizado.to_excel(nombre_archivo_excel, index=False)

    return df_sell_out_actualizado

In [127]:
actualizar_consolidado_y_guardar_excel(datos, "tabla_sell_out_consolidado_22.xlsx")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_35956\1800045614.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35956\1800045614.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(


,Mes,año,cliente,Unidades,NumMes
0,ABR,2023,PROMOTORA DE COMERCIO SOCIAL,817,4
1,ABR,2024,PROMOTORA DE COMERCIO SOCIAL,738,4
2,AGO,2022,PROMOTORA DE COMERCIO SOCIAL,438,8
3,AGO,2023,PROMOTORA DE COMERCIO SOCIAL,688,8
4,AGO,2024,PROMOTORA DE COMERCIO SOCIAL,736,8
...,...,...,...,...,...
70,SEP,2024,FARMATODO COLOMBIA SA,713,9
71,MAR,2025,FARMATODO COLOMBIA SA,636,3
72,MAR,2025,PROMOTORA DE COMERCIO SOCIAL,533,3
73,MAR,2025,MAKRO SUPERMAYORISTA SAS,2556,3


## TABLA SELL OUT CONSOLIDADO CIUDAD - DESCRIPCIÓN

In [137]:
import pandas as pd

def actualizar_consolidado_sell_out_ciudades_y_guardar_excel(datos, nombre_archivo_excel):
    
    _, df_promotora = procesar_promotora(datos['tabla_cliente_promotora'])

    _, df_makro = procesar_makro(datos['tabla_cliente_makro'])

    _, df_farmatodo = procesar_farmatodo(datos['tabla_cliente_farmatodo'])

    df_sell_out_actualizado_ciudades = pd.concat([
        datos['tabla_sell_out_ciudades_consolidado'],
        df_promotora,
        df_makro,
        df_farmatodo
    ], ignore_index=True)

    df_sell_out_actualizado_ciudades.to_excel(nombre_archivo_excel, index=False)

    return df_sell_out_actualizado_ciudades

In [138]:
actualizar_consolidado_sell_out_ciudades_y_guardar_excel(datos, "tabla_sell_out_ciudad_consolidado_110.xlsx")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_35956\3542092328.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(
C:\Users\Usuario\AppData\Local\Temp\ipykernel_35956\3542092328.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  archivo_entrada_promotora[['Tienda', 'Ciudad']] = archivo_entrada_promotora['Nomb. Dependencia'].apply(


,EAN,Mes,año,cliente,Tienda,Ciudad,Descripcion,Unidades,NumMes
0,7.71e+12,ABR,2023.0,PROMOTORA DE COMERCIO SOCIAL,BAZAR CARULLA BOGOTA,BOGOTA,Splash EL JARDIN DE EVA antmosq.x135ml,26.0,4
1,7.71e+12,ABR,2023.0,PROMOTORA DE COMERCIO SOCIAL,BAZAR CARULLA BOGOTA,BOGOTA,Splash EL JARDIN DE EVA lavan.manzx135ml,41.0,4
2,7.71e+12,ABR,2023.0,PROMOTORA DE COMERCIO SOCIAL,CARULLA ARRECIFE,SANTA MARTA,Splash EL JARDIN DE EVA antmosq.x135ml,2.0,4
3,7.71e+12,ABR,2023.0,PROMOTORA DE COMERCIO SOCIAL,CARULLA EL TESORO,MEDELLIN,Aceite EL JARDIN DE EVA bifasico 400ml,1.0,4
4,7.71e+12,ABR,2023.0,PROMOTORA DE COMERCIO SOCIAL,CARULLA EL TESORO,MEDELLIN,Kit EL JARDIN DE EVA relajate x4 unds,2.0,4
...,...,...,...,...,...,...,...,...,...
16846,7.71e+12,ABR,2025.0,FARMATODO COLOMBIA SA,Usaquen,Bogota,Splash Buenas noches Eva Lavanda y Manzanilla ...,2.0,4
16847,7.71e+12,ABR,2025.0,FARMATODO COLOMBIA SA,Variante Chia,Chia,Splash Buenas noches Eva Lavanda y Manzanilla ...,1.0,4
16848,7.71e+12,ABR,2025.0,FARMATODO COLOMBIA SA,Versalles,Cali,Splash Buenas noches Eva Lavanda y Manzanilla ...,1.0,4
16849,7.71e+12,ABR,2025.0,FARMATODO COLOMBIA SA,Villa Carolina,Barranquilla,Splash Buenas noches Eva Lavanda y Manzanilla ...,1.0,4
